In [1]:
import pandas as pd

from evidently import ColumnMapping
from evidently.report import Report
from evidently.test_suite import TestSuite

from evidently.metric_preset import DataDriftPreset
from evidently.metric_preset import DataQualityPreset
from evidently.metric_preset import RegressionPreset
from evidently.metric_preset import ClassificationPreset
from evidently.metric_preset import TargetDriftPreset
from evidently.metric_preset import TextOverviewPreset

from evidently.metrics import *

from evidently.test_preset import NoTargetPerformanceTestPreset
from evidently.test_preset import DataStabilityTestPreset
from evidently.test_preset import DataQualityTestPreset
from evidently.test_preset import DataDriftTestPreset

from evidently.tests import *

from evidently.tests.base_test import generate_column_tests
from evidently.metrics.base_metric import generate_column_metrics

/home/vishesh/anaconda3/envs/alibidetect/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-22 16:37:27.736566: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-22 16:37:27.806555: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-22 16:37:28.222030: W tensorflow/compiler/xla/stream_executor/

In [2]:
import nltk
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package words to /home/vishesh/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vishesh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/vishesh/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [7]:
perturbation = "gaussiannoise"
strength = 1
ref_path = "/home/vishesh/Desktop/datasets/fleurs/embeddings/train_decoder_embeddings.csv"
# cur_path = f"/home/vishesh/Desktop/datasets/fleurs/embeddings/perturbations/{perturbation}/test_results_{perturbation}_{strength}.csv"
cur_path = f"/home/vishesh/Desktop/datasets/fleurs/embeddings/test_decoder_embeddings.csv"

In [8]:
reference = pd.read_csv(ref_path)
current = pd.read_csv(cur_path)
current.drop_duplicates(subset=['Labels'], inplace=True)
reference.drop_duplicates(subset=['Labels'], inplace=True)

columns_to_del = [f"Embedding_{i}" for i in range (1,769)]
reference.drop(columns=columns_to_del, inplace=True)
current.drop(columns=columns_to_del, inplace=True)

In [9]:
column_mapping = ColumnMapping(
    text_features=['Predictions', 'Labels']
)

In [19]:
import json

data_drift_report = Report(metrics=[
    DataDriftPreset(num_stattest='ks', cat_stattest='psi', num_stattest_threshold=0.2, cat_stattest_threshold=0.2),
])

data_drift_report.run(reference_data=reference, current_data=current, column_mapping=column_mapping)
json.loads(data_drift_report.json())['metrics']

[{'metric': 'DatasetDriftMetric',
  'result': {'drift_share': 0.5,
   'number_of_columns': 5,
   'number_of_drifted_columns': 3,
   'share_of_drifted_columns': 0.6,
   'dataset_drift': True}},
 {'metric': 'DataDriftTable',
  'result': {'number_of_columns': 5,
   'number_of_drifted_columns': 3,
   'share_of_drifted_columns': 0.6,
   'dataset_drift': True,
   'drift_by_columns': {'Duration': {'column_name': 'Duration',
     'column_type': 'num',
     'stattest_name': 'K-S p_value',
     'stattest_threshold': 0.2,
     'drift_score': 0.031594709265934356,
     'drift_detected': True,
     'current': {'small_distribution': {'x': [3.96,
        6.494,
        9.027999999999999,
        11.562,
        14.096,
        16.63,
        19.163999999999998,
        21.698,
        24.232,
        26.766,
        29.3],
       'y': [0.06652384710790395,
        0.10936971473672345,
        0.124027511557109,
        0.048483481790506246,
        0.02480550231142182,
        0.011275228323373552,
 

In [17]:
# text_overview_report = Report(metrics=[
#     TextOverviewPreset(column_name="Predictions")
# ])

# text_overview_report.run(reference_data=reference, current_data=current, column_mapping=column_mapping)
# text_overview_report